In [7]:
from gensim.models import KeyedVectors
EMBEDDING_FILE = '../word2vec/GoogleNews-vectors-negative300.bin.gz'

In [6]:
import numpy as np 
import pandas as pd 

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Merge
import keras.backend as K
from keras.optimizers import Adadelta, Adagrad

from nltk import word_tokenize

In [8]:
import itertools
import datetime
from time import time

import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
data = pd.read_csv("../datasets/quora.csv")


In [10]:
train = data[:1000].copy(deep=True)
test = data[1000:2000].copy(deep=True)

In [11]:
def tokenize(s):
    s = s.lower()
    if s[-1] == '?':
        s = s[:-1]
        
    tokens = word_tokenize(s)
    return tokens

## Phrase to word indice vector
- Use only word found in pretrained word vector and ignore others

In [14]:
# Prepare embedding
vocabulary = dict()
inverse_vocabulary = ['<unk>'] 
if True:
    word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [13]:
data.head()

,q1,q2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,1
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [17]:
# Iterate over the questions only of both training and test datasets
for dataset in [train, test]:
    for index, row in dataset.iterrows():
        #print(row)

        for question in ['q1', 'q2']:

            q2n = []
            for word in tokenize(row[question]):
                
                if word not in word2vec.vocab:
                    continue
            
                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])

            # Replace questions as word to question as number representation
            dataset.set_value(index, question, q2n)


## Embedding matrix

In [18]:
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)
        
print(embeddings.shape)

(5501, 300)


In [19]:
questions_cols = ['q1', 'q2']
max_length = 150

X_train = train[questions_cols]
Y_train = train['is_duplicate']
X_train = {'left': pad_sequences(X_train.q1, maxlen = max_length), 
           'right': pad_sequences(X_train.q2, maxlen = max_length)}
Y_train = Y_train.values

X_test = test[questions_cols]
Y_test = test['is_duplicate']
X_test = {'left': pad_sequences(X_test.q1, maxlen = max_length), 
           'right': pad_sequences(X_test.q2, maxlen = max_length)}
Y_test = Y_test.values


In [20]:
X_train

{'left': array([[   0,    0,    0, ...,   10,    8,   11],
        [   0,    0,    0, ...,    3,   12,   13],
        [   0,    0,    0, ...,   29,   30,   31],
        ..., 
        [   0,    0,    0, ...,  793,  318,  299],
        [   0,    0,    0, ..., 2532,  681, 3261],
        [   0,    0,    0, ...,   97, 1901, 1902]], dtype=int32),
 'right': array([[   0,    0,    0, ...,    8,    9,   10],
        [   0,    0,    0, ...,    3,   13,   20],
        [   0,    0,    0, ...,   34,   35,   36],
        ..., 
        [   0,    0,    0, ...,  373, 1550, 3520],
        [   0,    0,    0, ..., 2532,  681, 3261],
        [   0,    0,    0, ..., 2237,  891,    8]], dtype=int32)}

# Siamese + Manhattan error

In [21]:
# Model variables
n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 10

# exp ( - |a - b| (L1))
def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# The visible layer
left_input = Input(shape=(max_length,), dtype='int32')
right_input = Input(shape=(max_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), 
                            embedding_dim, 
                            weights=[embeddings], 
                            input_length=max_length, 
                            trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Merge(mode=lambda x: exponent_neg_manhattan_distance(x[0], x[1]), 
                        output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

# Start training
training_start_time = time()

malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=n_epoch,
                            validation_data=([X_test['left'], X_test['right']], Y_test))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))
                                                       
print("Save model")                                                
malstm.save("../output/manhattan_siamese.h5")

/Users/canh/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:34: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/canh/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:48: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 10s 10ms/step - loss: 0.2547 - acc: 0.6270 - val_loss: 0.2367 - val_acc: 0.6600
Epoch 2/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2454 - acc: 0.6370 - val_loss: 0.2323 - val_acc: 0.6600
Epoch 3/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2366 - acc: 0.6430 - val_loss: 0.2284 - val_acc: 0.6600
Epoch 4/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2285 - acc: 0.6560 - val_loss: 0.2252 - val_acc: 0.6620
Epoch 5/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2208 - acc: 0.6610 - val_loss: 0.2222 - val_acc: 0.6670
Epoch 6/10
1000/1000 [==============================] - 8s 8ms/step - loss: 0.2133 - acc: 0.6710 - val_loss: 0.2196 - val_acc: 0.6710
Epoch 7/10
1000/1000 [==============================] - 9s 9ms/step - loss: 0.2063 - acc: 0.6760 - val_loss: 0.2173 - val_acc: 0.6730
Epoch 8/10
1

# Dense layer

In [2]:
from keras.layers import Concatenate, Dense

In [ ]:
# Model variables
n_hidden = 50
n_dense = 100
#gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 1


# The visible layer
left_input = Input(shape=(max_length,), dtype='int32')
right_input = Input(shape=(max_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), 
                            embedding_dim, 
                            weights=[embeddings], 
                            input_length=max_length, 
                            trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# concatenate outputs
merge_output = Concatenate(axis=-1)([left_ouput, right_output])

#Dense layer
dense_layer = Dense(units=n_dense, activation="relu")(merge_output)

# Pack it all up into a model
model = Model([left_input, right_input], [dense_layer])

# Adadelta optimizer, with gradient clipping by norm
optimizer = Adagrad()

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Start training
training_start_time = time()

model.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=n_epoch,
                            validation_data=([X_test['left'], X_test['right']], Y_test))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))
                                                       
print("Save model")                                                
model.save("../output/dense_layer_siamese.h5")

In [ ]:
# Plot accuracy
plt.plot(malstm_trained.history['acc'])
plt.plot(malstm_trained.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss
plt.plot(malstm_trained.history['loss'])
plt.plot(malstm_trained.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()